In [2]:
import matplotlib.pyplot as plt
import pytorch3d
import torch
import numpy as np
import imageio

from starter.utils import get_device, get_mesh_renderer, load_cow_mesh

### 360-degree render

In [4]:
device = get_device()
cow_path="data/cow.obj"
image_size=256
color=[0.7, 0.7, 1]

In [3]:
renderer = get_mesh_renderer(image_size=image_size)

In [4]:
# Get the vertices, faces, and textures.
vertices, faces = load_cow_mesh(cow_path)
vertices = vertices.unsqueeze(0)  # (N_v, 3) -> (1, N_v, 3)
faces = faces.unsqueeze(0)  # (N_f, 3) -> (1, N_f, 3)
textures = torch.ones_like(vertices)  # (1, N_v, 3)
textures = textures * torch.tensor(color)  # (1, N_v, 3)
mesh = pytorch3d.structures.Meshes(
    verts=vertices,
    faces=faces,
    textures=pytorch3d.renderer.TexturesVertex(textures),
)
mesh = mesh.to(device)

In [5]:
# Prepare the camera:
cameras = pytorch3d.renderer.FoVPerspectiveCameras(
    R=torch.eye(3).unsqueeze(0), T=torch.tensor([[0, 0, 3]]), fov=60, device=device
)

# Place a point light in front of the cow.
lights = pytorch3d.renderer.PointLights(location=[[0, 0, -3]], device=device)

In [7]:
num_views = 36
R, T = pytorch3d.renderer.look_at_view_transform(
    dist=3,
    elev=0,
    azim=np.linspace(-180, 180, num_views, endpoint=False),
)
print("R", R.shape)
print("T", T.shape)

R torch.Size([36, 3, 3])
T torch.Size([36, 3])


In [8]:
many_cameras = pytorch3d.renderer.FoVPerspectiveCameras(
    R=R,
    T=T,
    device=device
)

In [9]:
renderer = get_mesh_renderer(image_size=512)

In [10]:
images = renderer(mesh.extend(num_views), cameras=many_cameras, lights=lights)

In [11]:
images.shape

torch.Size([36, 512, 512, 4])

In [12]:
my_images = [(img[..., :3].cpu().numpy() * 255).astype(np.uint8) for img in images]  # List of images [(H, W, 3)]

In [13]:
duration = 1000 // 30  # Convert FPS (frames per second) to duration (ms per frame)
imageio.mimsave('my_gif.gif', my_images, duration=duration, loop=0)  # loop=0 for infinite loop

### Dolly zoom

In [15]:
torch.linspace(120, 5, num_views)

tensor([120.0000, 116.7143, 113.4286, 110.1429, 106.8571, 103.5714, 100.2857,
         97.0000,  93.7143,  90.4286,  87.1429,  83.8571,  80.5714,  77.2857,
         74.0000,  70.7143,  67.4286,  64.1429,  60.8571,  57.5714,  54.2857,
         51.0000,  47.7143,  44.4286,  41.1429,  37.8571,  34.5714,  31.2857,
         28.0000,  24.7143,  21.4286,  18.1429,  14.8571,  11.5714,   8.2857,
          5.0000])

In [2]:
import torch

In [3]:
fovs = torch.linspace(25, 100, 10)

In [4]:
fovs[0], fovs[0].item()

(tensor(25.), 25.0)

### Constructing a tetrahedron

In [31]:
vertices = torch.tensor([[0,1,0], 
                         [3, 0, 0],
                         [0, 2, 0],
                         [1, 1, 3]], dtype=torch.float32, device=device).unsqueeze(0)  

faces = torch.tensor([[0, 1, 2],
                      [0, 1, 3],
                      [0, 2, 3],
                      [1, 2, 3]], dtype=torch.int64, device=device).unsqueeze(0)

In [32]:
color = torch.tensor([[0.2, 0.5, 1.0]], device=device)
textures = torch.ones_like(vertices)
textures = textures * torch.tensor(color)

mesh = pytorch3d.structures.Meshes(
    verts=vertices,
    faces=faces,
    textures=pytorch3d.renderer.TexturesVertex(textures)
)
mesh = mesh.to(device)

/tmp/ipykernel_297909/4186888042.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  textures = textures * torch.tensor(color)


In [33]:
# Prepare the camera:
cameras = pytorch3d.renderer.FoVPerspectiveCameras(
    R=torch.eye(3).unsqueeze(0), T=torch.tensor([[0, 0, 3]]), fov=60, device=device
)

# Place a point light in front of the cow.
lights = pytorch3d.renderer.PointLights(location=[[0, 0, -3]], device=device)

In [34]:
num_views = 36
R, T = pytorch3d.renderer.look_at_view_transform(
    dist=5,
    elev=30,
    azim=np.linspace(-180, 180, num_views, endpoint=False),
)

In [35]:
many_cameras = pytorch3d.renderer.FoVPerspectiveCameras(
    R=R,
    T=T,
    device=device
)

In [36]:
renderer = get_mesh_renderer(image_size=512)
images = renderer(mesh.extend(num_views), cameras=many_cameras, lights=lights)
my_images = [(img[..., :3].cpu().numpy() * 255).astype(np.uint8) for img in images]  # List of images [(H, W, 3)]
duration = 1000 // 30  # Convert FPS (frames per second) to duration (ms per frame)
imageio.mimsave('tetrahedron.gif', my_images, duration=duration, loop=0)  # loop=0 for infinite loop

### Constructing a cube

In [ ]:
vertices = torch.tensor([[0, 0, 0], #0 
                         [1, 0, 0], #1
                         [1, 0, -1], #2
                         [0, 0, -1], #3
                         [0, 1, 0], #4
                         [1, 1, 0], #5
                         [1, 1, -1], #6
                         [0, 1, -1]], dtype=torch.float32, device=device).unsqueeze(0) #7

# mind the winding order, when looking from outside the cube vertices should be counter-clockwise
faces = torch.tensor([[0, 1, 2], [0, 2, 3],
                      [0, 5, 4], [0, 1, 5],
                      [0, 7, 4], [0, 3, 7],
                      [1, 2, 5], [2, 6, 5],
                      [2, 6, 3], [7, 3, 6],
                      [4, 7, 5], [7, 6, 5]], dtype=torch.int64, device=device).unsqueeze(0)

In [69]:
color = torch.tensor([[0.2, 0.5, 1.0]], device=device)
textures = torch.ones_like(vertices)
textures = textures * torch.tensor(color)

mesh = pytorch3d.structures.Meshes(
    verts=vertices,
    faces=faces,
    textures=pytorch3d.renderer.TexturesVertex(textures)
)
mesh = mesh.to(device)

/tmp/ipykernel_297909/4186888042.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  textures = textures * torch.tensor(color)


In [70]:
# Prepare the camera:
cameras = pytorch3d.renderer.FoVPerspectiveCameras(
    R=torch.eye(3).unsqueeze(0), T=torch.tensor([[0, 0, 3]]), fov=60, device=device
)

# Place a point light in front of the cow.
lights = pytorch3d.renderer.PointLights(location=[[0, 0, -3]], device=device)

In [71]:
num_views = 36
R, T = pytorch3d.renderer.look_at_view_transform(
    dist=5,
    elev=30,
    azim=np.linspace(-180, 180, num_views, endpoint=False),
)

In [72]:
many_cameras = pytorch3d.renderer.FoVPerspectiveCameras(
    R=R,
    T=T,
    device=device
)

In [73]:
renderer = get_mesh_renderer(image_size=512)
images = renderer(mesh.extend(num_views), cameras=many_cameras, lights=lights)
my_images = [(img[..., :3].cpu().numpy() * 255).astype(np.uint8) for img in images]  # List of images [(H, W, 3)]
duration = 1000 // 30  # Convert FPS (frames per second) to duration (ms per frame)
imageio.mimsave('cube.gif', my_images, duration=duration, loop=0)  # loop=0 for infinite loop

### Re-texturing a mesh

In [118]:
vertices, faces = load_cow_mesh(cow_path)
vertices = vertices.unsqueeze(0)
faces = faces.unsqueeze(0)  

vertices = vertices.to(device)
faces = faces.to(device)    

In [119]:
colour1 = torch.tensor([1.0, 0.0, 0.0], device=device)  
colour2 = torch.tensor([0.0, 0.0, 1.0], device=device)

In [120]:
z_coords = vertices[0, :, 2]
z_min = z_coords.min()
z_max = z_coords.max()

In [121]:
alpha = (z_coords - z_min) / (z_max - z_min)

In [122]:
vertex_colors = alpha[:, None] * colour2 + (1 - alpha[:, None]) * colour1
vertex_colors = vertex_colors.unsqueeze(0)  

In [123]:
mesh = pytorch3d.structures.Meshes(
    verts=vertices,
    faces=faces,
    textures=pytorch3d.renderer.TexturesVertex(vertex_colors)
).to(device)

In [124]:
num_views = 36
R, T = pytorch3d.renderer.look_at_view_transform(
    dist=3, elev=0, azim=np.linspace(-180, 180, num_views, endpoint=False)
)
many_cameras = pytorch3d.renderer.FoVPerspectiveCameras(R=R, T=T, device=device)
images = renderer(mesh.extend(num_views), cameras=many_cameras, lights=lights)
my_images = [(img[..., :3].cpu().numpy() * 255).astype(np.uint8) for img in images]
imageio.mimsave('gradient_cow.gif', my_images, duration=1000//30, loop=0)